In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# %pip install bitsandbytes --prefer-binary --extra-index-url https://jllllll.github.io/bitsandbytes-windows-webui
# %pip install soundfile
%pip install -q diffusers

#option to text/tospeech
# %pip install torch torchaudio transformers soundfile





In [ ]:
import torch
print(torch.version.cuda)              # should return something like "11.8"
print(torch.cuda.is_available())       # should return True
print(torch.cuda.get_device_name(0))   # should return your GPU name

In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

In [ ]:
load_dotenv(override=True)

hf_token = os.getenv('HUGGING_FACE_CLAIRE')
if hf_token:
    print(f"Hugging Face Key exists and begins {hf_token[:8]}")
else:
    print(f"Hugging Face Key does not exists.")

login(token=hf_token, add_to_git_credential=True)

In [ ]:
from huggingface_hub import whoami
print(whoami())

In [ ]:

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

### Loading image from a pretrained model

In [ ]:

from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.float16,
    device_map="balanced"  # ✅ required for Flux
)

generator = torch.Generator(device="cuda").manual_seed(0)
prompt = "A futuristic class full of students learning AI coding in the surreal style of Salvador Dali"

# Generate the image using the GPU
image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=64,
    height=192,
    width=340,
    generator=generator
).images[0]

image.save("surreal2.png")

In [ ]:
from IPython.display import display
from PIL import Image

# Display the image inline
display(image)

### Testing text to speech models

In [ ]:
#This is not working because huggingface is not supporting loading this model by code anymore - 07/17/25

# from transformers import pipeline
# from datasets import load_dataset
# import soundfile as sf
# import torch

# synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cuda')

# embeddings_dataset = load_dataset("matthijs/cmu-arctic-xvectors", split="validation")

# speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# speech = synthesiser("Hi to an artificial intelligence engineer on the way to mastery!", forward_params={"speaker_embeddings": speaker_embedding})

# sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import torch
import soundfile as sf
import numpy as np

# Load models
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to("cuda")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to("cuda")

# Load pre-extracted xvector (you already have these)
xvector_path = r'D:\AI\ai-llm-course\Week3\spkrec-xvect\cmu_us_bdl_arctic-wav-arctic_a0007.npy'
xvector = np.load(xvector_path)
speaker_embedding = torch.tensor(xvector).unsqueeze(0).to("cuda")

# Prepare input
text = "Hello to an artificial intelligence engineer on the way to mastery!"
inputs = processor(text=text, return_tensors="pt").to("cuda")

# Generate speech hidden states
with torch.no_grad():
    speech = model.generate_speech(inputs["input_ids"], speaker_embedding, vocoder=vocoder)

# Save to WAV
sf.write("output.wav", speech.cpu().numpy(), samplerate=16000)


In [ ]:
from IPython.display import Audio

# Play the generated audio
Audio("output.wav")

### Loading image from another pretrained model - less quality/faster

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sd-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

prompt = "A futuristic class full of students learning AI coding in a surreal style"
image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0]

In [ ]:
from IPython.display import display
from PIL import Image

# Display the image inline
display(image)